# Pregunta 2: Schnorr signature

## Imports

In [204]:
import hashlib
from random import randint, random

## Helper functions

In [205]:
# Functions extracted from T2
# exp_mod modified to allow a negative exponent

def exp_mod(a: int, b: int, n: int) -> int:
    """
    Modular exponentation algorithm

    Args:
        a (int): number >= 0
        b (int): number
        n (int): number > 0

    Returns:
        int: a**b mod n
    """
    a = a % n
    if n == 1:
        return 0
    if b == 0:
        return 1
    if a == 0:
        return 0
    c = 1
    if b < 0:
        a = inverso(a, n)
        b = -b
    while b > 0:
        if ((b % 2) != 0):
            c = (c * a) % n
        b //= 2
        a = (a * a) % n
    return c

def alg_ext_euclides(a: int, b: int) -> tuple[int, int, int]:
    """
    Extended euclidean algorithm

    Args:
        a (int): number > 0
        b (int): a >= b >=

    Returns:
        tuple[int, int, int]: (GCD(a,b), s, t) greatest common divisor GCD(a, b) of a and b, and
            integers s and t: MCD(a, b) = s*a + t*b
    """
    # 1 and 0 are always going to be the factors of the last equations
    s_prev, t_prev, = 1, 0,
    s, t = 0, 1
    while b != 0:
        q = a // b
        a, b = b, a % b
        if b != 0:
            s_prev, s = s, s_prev - q * s
            t_prev, t = t, t_prev - q * t
    return a, s, t

def inverso(a: int, n: int) -> int:
    """
    Modular inverse

    Args:
        a (int): number >= 1
        n (int): number >= 2, relative prime of a

    Returns:
        int: modular inverse of a in mod n    
    """
    x, y, _ = alg_ext_euclides(a, n)
    if x == 1:
        return y % n
    return None


## Read files

In [206]:
def read_grupo(pathname="grupo.txt"):
    """
    Loads global constants p, g, and q from file

    Arguments:
        pathname (str): path to file

    Returns:
        tuple[int]: values p, g, and q extracted from file.
    """
    with open(pathname, 'r') as grupo_file:
        grupo = grupo_file.readlines()
        p = int(grupo[0].strip(), 16)
        g = int(grupo[1].strip(), 16)
        q = int(grupo[2].strip(), 16)
    return p, g, q

def read_private_key(pathname="private_key.txt"):
    """
    Loads private key from file

    Arguments:
        pathname (str): path to file

    Returns:
        int: private key extracted from file.
    """
    with open(pathname, 'r') as private_key_file:
        private_key = int(private_key_file.readline().strip(), 16)
    return private_key

def read_public_key(pathname="public_key.txt"):
    """
    Loads public key from file

    Arguments:
        pathname (str): path to file

    Returns:
        int: public key extracted from file.
    """
    with open(pathname, 'r') as public_key_file:
        public_key = int(public_key_file.readline().strip(), 16)
    return public_key


## Implementation

In [207]:
def generar_clave_ElGamal():
    """
    Generates a public and private key with ElGamal protocol, with base values extracted from
    grupo.txt file.

    Stores the private key in private_key.txt file and the public key in public_key.txt file
    """
    p, g, q = read_grupo()

    # Generate the private key
    x = randint(1, q - 1)
    y = exp_mod(g, x, p)

    # Output keys
    with open('private_key.txt', 'w') as pk_file:
        pk_file.write(hex(x)[2:].upper())
    with open('public_key.txt', 'w') as sk_file:
        sk_file.write(hex(y)[2:].upper())

def firmar_Schnorr(m: str) -> tuple[int, int]:
    """
    Signs a message with Schnorr protocol, using the private key stored in private_key.txt file, and
    variables stored in grupo.txt file.

    Arguments:
        m (str): Message to be signed
    
    Returns:
        tuple[int, int]: message Schnorr signature in the form of (e, s)
    """
    p, g, q = read_grupo()
    x = read_private_key()
    k = randint(1, q - 1)

    r = exp_mod(g, k, p)
    e = md5(str(r) + m)
    s = k - x * e
    
    return (e, s)

def verificar_firma_Schnorr(m: str, firma: tuple[int, int]) -> bool:
    """
    Verifies if a Schnorr signature corresponds to the specified message, using the public key
    stored in public_key.txt file, and variables stored in grupo.txt file.

    Arguments:
        m (str): Message to be verified
        firma (tuple[int,int]): Signature to be verified

    Returns:
        bool: True if the signature corresponds to the message, False otherwise
    """
    p, g, _ = read_grupo()
    e, s = firma
    y = read_public_key()

    r_prime = (exp_mod(g, s, p) * exp_mod(y, e, p)) % p
    return md5(str(r_prime) + m) == e


def md5(m: str) -> int:
    """
    Hashes a message using md5 function

    Arguments:
        m (str): Message to be hashed

    Returns:
        int: Hash of the message in the form of int
    """
    return int.from_bytes(hashlib.md5(bytes(m, 'utf-8')).digest(), 'big')


## Tests

### Generate key

In [208]:
if __name__ == '__main__':
    # Generate private_key.txt and public_key.txt
    generar_clave_ElGamal()


### Single test

In [209]:
if __name__ == '__main__':
    # Generate private_key.txt and public_key.txt
    generar_clave_ElGamal()

    # Firmar mensaje
    test_message = "Hola Mundo!"
    firma = firmar_Schnorr(test_message)

    # Validar firma
    resultado = verificar_firma_Schnorr(test_message, firma)
    print(resultado)

True


### Multi test

In [210]:
if __name__ == '__main__':
    for i in range(1, 1000):
        # Generate private_key.txt and public_key.txt
        generar_clave_ElGamal()

        # Generate random message 
        current_message = ''.join([chr(randint(0, 127)) for _ in range(randint(20, 10000))])

        # Firmar mensaje
        # Corromper firma con cierta probabilidad
        if random() < 0.25:
            integrity = False
            # flip message
            firma = firmar_Schnorr(current_message[::-1])
        else:
            integrity = True
            firma = firmar_Schnorr(current_message)

        # Validar firma
        resultado = verificar_firma_Schnorr(current_message, firma)

        assert(resultado == integrity)
